In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.utils import create_ordinal_cost_matrix


from ordinaloss.engine.model_engine import OrdinalEngine
from ordinaloss.utils.datasets import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.utils.loss_utils
loaded ordinaloss.utils.datasets


In [101]:
# train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
# test_transform = create_transform_pipeline() #Without color jitter for test pipeline

# train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
# val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
# test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
# auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

# train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
# test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

loaders, _, _ = data_load("../datasets/kneeKL224/", 16)
train_loader = loaders["train"]
test_loader = loaders["test"]

loaded ordinaloss.utils.loss_utils


In [112]:
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

In [103]:
create_ordinal_cost_matrix(5)

tensor([[20.,  1.,  2.,  3.,  4.],
        [ 1., 20.,  1.,  2.,  3.],
        [ 2.,  1., 20.,  1.,  2.],
        [ 3.,  2.,  1., 20.,  1.],
        [ 4.,  3.,  2.,  1., 20.]])

In [117]:
from ordinaloss.utils.loss_utils import GirlsLoss, SinimLoss
model = classification_model_vgg("vgg16", num_classes = 5)

def my_lambda_scheduler(epoch):
    return (0.9 ** (epoch // 5))


# engine = OrdinalEngine(model, 
#                        loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


# engine = OrdinalEngine(model, 
#                        loss_fn = SinimLoss(), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=Adam, lr = 1.0e-3, weight_decay=5.0e-2)                       

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 10.76 GiB total capacity; 5.11 GiB already allocated; 346.44 MiB free; 5.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [110]:
torch.cuda.empty_cache()

In [111]:
engine.train(train_loader, test_loader, n_epochs = 1)

Training, epoch 0:   0%|          | 0/362 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 10.76 GiB total capacity; 5.00 GiB already allocated; 348.44 MiB free; 5.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF